丟一些測試的代碼

In [39]:
import pandas as pd
import numpy as np
import cv2
import os
from PIL import Image
import math
import matplotlib.pyplot as plt

In [ ]:
def rgb2hsv(color):
    color_list=color.tolist()
    b,g,r=color_list[0],color_list[1],color_list[2]
    r, g, b = r/255.0, g/255.0, b/255.0
    mx, mn = max(r, g, b), min(r, g, b)
    m = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        if g >= b:
            h = ((g-b)/m)*60
        else:
            h = ((g-b)/m)*60 + 360
    elif mx == g:
        h = ((b-r)/m)*60 + 120
    elif mx == b:
        h = ((r-g)/m)*60 + 240
    if mx == 0:
        s = 0
    else:
        s = m/mx
    v = mx
    return h, s, v


資料庫
===

In [3]:
flower_name_dict={
'tree poppy': '樹罌粟',
'alpine sea holly': '海濱刺芹',
'balloon flower': '桔梗',
'bishop of llandaff': '大麗花',
'black-eyed susan': '黑心金光菊',
'blanket flower': '天人菊',
'californian poppy': '花菱草',
'daisy': '雛菊',
'giant white arum lily': '海芋',
'globe thistle': '藍刺頭',
'iris': '鳶尾花',
'lilac hibiscus' :'朱槿',
'orange dahlia' :'大麗花',
'osteospermum': '藍眼菊',
'pink-yellow dahlia':'',
'poinsettia':'聖誕紅',
'purple coneflower': '紫錐花',
'sunflower': '向日葵',
'thorn apple' :'曼陀羅',
'tree mallow': '花葵'
    
}

In [5]:
import Flower
#Flower.show_match('C:/flowerpic/jpeg-224x224/val/iris/12769.jpeg','C:/flowerpic/jpeg-224x224/val/iris/16301.jpeg')


爬蟲
===

In [53]:
import requests
import re
import time
import numpy as np
from selenium import webdriver


driver=webdriver.Chrome()
driver.get("https://m.coa.gov.tw/Transaction/AgriculturalProduct/Index?TcType=N06")

#MarketId
#class="multiselect-native-select"
marked = driver.find_element_by_class_name("multiselect-native-select")
marked.click()
#select = Select(driver.find_element_by_class_name("custom-select"))
all_select=driver.find_element_by_class_name("checkbox")
all_select.click()


flower_name = driver.find_element_by_id("ByKeyword")
flower_name.send_keys("康乃馨")

button = driver.find_element_by_id("btn-search")
button.click()





In [85]:
import pandas as pd
title={
    "date":[],
    'type':[],
    'id':[],
    'name':[],
    'opt':[],
    'place':[],
    'price_up':[],
    'price_mid':[],
    'price_low':[],
    'peice':[],}

flower_price=pd.DataFrame(title)
for i in range(2,22):
    x=(driver.find_element_by_xpath(f"/html/body/div[1]/main/div[2]/div/section[2]/table/tbody/tr[{i}]").text)

    X=x.split(" ")
    data={
    "date":X[0],
    'type':X[1],
    'id':X[2],
    'name':X[3],
    'opt':X[4],
    'place':X[5],
    'price_up':X[6],
    'price_mid':X[7],
    'price_low':X[8],
    'peice':X[9]}
    flower_price=flower_price.append(data,ignore_index=True)
    
    

In [84]:
title={
    "date":[],
    'type':[],
    'id':[],
    'name':[],
    'opt':[],
    'place':[],
    'price_up':[],
    'price_mid':[],
    'price_low':[],
    'peice':[],}

flower_price=pd.DataFrame(title)

In [86]:
flower_price

,date,type,id,name,opt,place,price_up,price_mid,price_low,peice
0,112/10/13,花卉,FA800,康乃馨,多朵(混色),高雄市,50,50,50,50\n+2.0%\n60\n+0.0%
1,112/10/13,花卉,FA800,康乃馨,多朵(混色),台中市,45,45,45,45\n+21.6%\n60\n+0.0%
2,112/10/13,花卉,FA812,康乃馨,波斯桃紅,高雄市,40,40,40,40\n-\n30\n-
3,112/10/12,花卉,IA800,進口康乃馨,多朵混色,溪湖鎮,113,113,113,113\n+3.7%\n144\n+350.0%
4,112/10/12,花卉,IA800,進口康乃馨,多朵混色,台中市,114,114,109,"112.8\n+44.6%\n144\n+1,100.0%"
5,112/10/12,花卉,IA001,進口康乃馨,混合色,高雄市,134,78,77,90\n-29.1%\n138\n+130.0%
6,112/10/12,花卉,IA001,進口康乃馨,混合色,台中市,149,73,71,88\n-38.7%\n138\n+130.0%
7,112/10/12,花卉,IA800,進口康乃馨,多朵混色,高雄市,85,85,85,85\n+0.0%\n108\n-23.9%
8,112/10/12,花卉,FA800,康乃馨,多朵(混色),高雄市,49,49,49,49\n+6.5%\n60\n+0.0%
9,112/10/12,花卉,FA800,康乃馨,多朵(混色),台中市,37,37,37,37\n-14.9%\n60\n+0.0%


In [64]:


#x=(driver.find_element_by_xpath("/html/body/div[1]/main/div[2]/div/section[2]/table/tbody/tr[2]").text)
#/html/body/div[1]/main/div[2]/div/section[2]/table/tbody/tr[3]
x_2=(driver.find_element_by_xpath("/html/body/div[1]/main/div[2]/div/section[2]/table/tbody/tr[3]").text)

#/html/body/div[1]/main/div[2]/div/section[2]/table/tbody/tr[21]

邊緣檢測(?)
---


In [32]:
import cv2
import numpy as np
import imutils
img="flower_pic/tree poppy/12795.jpeg"
image=cv2.imread(img)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#高斯慮波
#v2.GaussianBlur（ SRC，ksize，sigmaX [，DST [，sigmaY [，borderType ] ] ] ）

blurred = cv2.GaussianBlur(gray, (7, 7), 0)

#minVal和maxVal
edged = cv2.Canny(blurred, 50, 200)
cv2.imshow("SIFT", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [33]:
( cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#print("I count {} coins in this image".format(len(cnts)))

coins = image.copy()

cv2.drawContours(coins, cnts, -1, (0, 255, 0), 2)
cv2.imshow("SIFT", coins)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
for (i, c) in enumerate(cnts):

    (x, y, w, h) = cv2.boundingRect(c)

    print("Part #{}".format(i + 1))

    coin = image[y:y + h, x:x + w]

    cv2.imshow("part", coin)

    mask = np.zeros(image.shape[:2], dtype = "uint8")

    ((centerX, centerY), radius) = cv2.minEnclosingCircle(c)

    cv2.circle(mask, (int(centerX), int(centerY)), int(radius), 255, -1)

    mask = mask[y:y + h, x:x + w]
    cv2.imshow("masked part",cv2.bitwise_and(coin, coin, mask = mask))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
  

Part #1
Part #2
Part #3
Part #4


ANN
===